In [2]:
%%capture
!pip install wandb -qqq

In [ ]:
import tensorflow.keras as keras
from google.colab import drive
import numpy as np
import cv2
import wandb
from wandb.keras import WandbCallback

MODELS_PATH = "/content/gdrive/My Drive/magshimim/camerona/models/"
BASE = "mask_base_model.h5"
BEST = "mask_best_model.h5"
DATASET_PATH_DRIVE = "/content/gdrive/My Drive/magshimim/camerona/dataset/mask/"
DATASET_PATH_COLAB = "/content/mask/"
VAL_TEXT = DATASET_PATH_COLAB+"val.txt"
TRAIN_TEXT = DATASET_PATH_COLAB+"train.txt"
TEST_TEXT = DATASET_PATH_COLAB+"test.txt"

drive.mount('/content/gdrive/')

In [ ]:
!ln -s "/content/gdrive/My Drive/magshimim/camerona/dataset/mask/" "/content/"

In [ ]:
!wandb login

In [10]:
def buildModel(lr = 0.001):
  input_layer = keras.layers.Input(shape=[64,64,3])

  #block1
  layer = keras.layers.Conv2D(32, (3,3), strides=(1,1), activation='relu', padding='same', name='block1_conv1')(input_layer)
  layer = keras.layers.MaxPool2D(2, name='block1_max1')(layer)

  #block2
  layer = keras.layers.Conv2D(16, (3,3), strides=(1,1), activation='relu', padding='same', name='block2_conv1')(layer)
  layer = keras.layers.MaxPool2D(2, name='block2_max1')(layer)

  #output block
  layer = keras.layers.Flatten()(layer)
  layer = keras.layers.Dense(16, activation="relu")(layer)
  layer = keras.layers.Dense(1, activation="sigmoid", name='output_layer')(layer)
 
  model = keras.Model(inputs=input_layer, outputs=layer)
  opt = keras.optimizers.Adam(learning_rate=lr)
  model.compile(optimizer = opt, loss = ['binary_crossentropy'], metrics=[keras.metrics.BinaryAccuracy()])
  print(model.summary())

  return model

In [5]:
def getData(file_path):
  f = open(file_path, 'r')
  data = f.read().split('\n')
  f.close()
  x = []
  y = []
  all = len(data)
  cnt = 1
  for line in data:
    if cnt%250 == 0:
      print("{0}/{1}".format(cnt, all))
    cnt += 1
    line_data = line.split(" ")
    line_data[0] = line_data[0].replace('\\', '/')
    try:
      img = cv2.imread(DATASET_PATH_COLAB+line_data[0])
      img = cv2.resize(img, (64,64))/255
    except:
      continue
    x.append(img)
    y.append(int(line_data[1]))
  x = np.array(x)
  y = np.array(y)
  return x,y

In [ ]:
#building and saving base model
model = buildModel()
model.save(MODELS_PATH+BASE)

In [ ]:
wandb.init(project="mask-classification")

In [14]:
#callbacks
best_callback = keras.callbacks.ModelCheckpoint(MODELS_PATH+BEST, save_best_only=True)
wandb_callback = WandbCallback()

In [ ]:
#loading data
print("loading validation data")
val_x, val_y = getData(VAL_TEXT)
print("loading train data")
train_x, train_y = getData(TRAIN_TEXT)

In [ ]:
model = buildModel(0.0001)
model.load_weights(MODELS_PATH+BEST)

In [ ]:
model.fit(train_x,
          train_y,
          epochs=24, 
          validation_data=(val_x, val_y),
          callbacks=[best_callback, wandb_callback])

In [ ]:
test_x, test_y = getData(TEST_TEXT)

In [ ]:
loss, acc = model.evaluate(x = test_x, y = test_y)

In [22]:
model.save(MODELS_PATH+BEST)